In [ ]:
# !git clone https://github.com/yao8839836/kg-bert.git
# %cd kg-bert

!pip install transformers==2.5.1
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Faile

In [ ]:
# !python run_bert_triple_classifier.py --task_name kg --do_train  --do_eval --do_predict --data_dir ./data/WN11 --bert_model bert-base-uncased --max_seq_length 20 --train_batch_size 32 \
# --learning_rate 5e-5 --num_train_epochs 3.0 --output_dir ./output_WN11/  --gradient_accumulation_steps 1 --eval_batch_size 512

08/20/2025 10:34:39 - INFO - __main__ -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
08/20/2025 10:34:39 - WARNING - pytorch_pretrained_bert.tokenization -   The pre-trained model you are loading is an uncased model but you have set `do_lower_case` to False. We are setting `do_lower_case=True` for you but you may want to check this behavior.
08/20/2025 10:34:39 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp6oerxbbl
100% 231508/231508 [00:00<00:00, 4171295.10B/s]
08/20/2025 10:34:39 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp6oerxbbl to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/20/2025 10:34:39 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import torch.nn.functional as F
from math import log

In [ ]:
df = pd.read_csv('./data/Irish_populated_places.csv')

In [ ]:
df.head()

,place,placeName,abstract,country,subject,broaderSubject,placeType,broaderPlaceType,population
0,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/Place,2479
1,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/Location,2479
2,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/PopulatedPlace,2479
3,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/Settlement,2479
4,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/Town,2479


In [ ]:
with open("data/Irish-Places/entities.txt","w", encoding="utf-8") as f:
  for entity_id in df['place']:
    f.write(f"{entity_id}\n")

In [ ]:
def uri_remove(uri):
  return str(uri).strip().replace("http://dbpedia.org/resource/", "").replace("http://dbpedia.org/ontology/","").replace("Category:","").replace("_", " ")

entity2text, entity2textlong = {}, {}
for idx, row in df.iterrows():
  entity_id = row['place']
  if entity_id not in entity2text:
    text = f"{row['placeName']}. "
    longtext = f"{row['abstract']} "
    longtext += f"Population: {row['population']}"
    entity2text[entity_id] = text
    entity2textlong[entity_id] = longtext


for c in df["country"].unique():
    if c not in entity2text:
      entity2text[c] = f"{uri_remove(c)}"
    if c not in entity2textlong:
      entity2textlong[c] = f"{uri_remove(c)}. A country."


for s in df["subject"].unique():
    if s not in entity2text:
        entity2text[s] = f"{uri_remove(s)}"
    if s not in entity2textlong:
        entity2textlong[s] = f"{uri_remove(s)}. A subject category."

# 3. Add subjects
for s in df["broaderSubject"].unique():
    if s not in entity2text:
        entity2text[s] = f"{uri_remove(s)}"
    if s not in entity2textlong:
        entity2textlong[s] = f"{uri_remove(s)}. A parent subject category."

# 4. Add city types
for t in df["placeType"].unique():
    if t not in entity2text:
        entity2text[t] = f"{uri_remove(t)}"
    if t not in entity2textlong:
        entity2textlong[t] = f"{uri_remove(t)}. A type of place."

# 5. Add broader place types
for bt in df["broaderPlaceType"].unique():
    if bt not in entity2text:
        entity2text[bt] = f"{uri_remove(bt)}"
    if bt not in entity2textlong:
        entity2textlong[bt] = f"{uri_remove(bt)}. A type of place."


with open("data/Irish-Places/entity2text.txt","w", encoding="utf-8") as f:
  for entity, text in entity2text.items():
    f.write(f"{entity}\t{text}\n")

with open("data/Irish-Places/entity2textlong.txt","w", encoding="utf-8") as f:
  for entity, textlong in entity2textlong.items():
    f.write(f"{entity}\t{textlong}\n")

In [ ]:
triples = pd.read_csv("data/triples.tsv", sep="\t", names=['head','relation','tail'])
unstructured_relations = {'dbo:abstract', 'dbo:populationTotal', 'rdfs:label'}
triples = triples[~triples['relation'].isin(unstructured_relations)]

In [ ]:
relations_dict = {"dbo:abstract":"abstract",
                  "dbo:country":"country",
                  "rdfs:label":"the name of the place",
                  "dbo:populationTotal":"population",
                  "dbo:type":"the type of the place",
                  "rdf:type":"an instance of a class",
                  "dcterms:subject":"category of the place",
                  "skos:broader":"parent category"
                  }

In [ ]:
with open("data/Irish-Places/relation2text.txt", "w", encoding="utf-8") as f:
    for r, desc in relations_dict.items():
        f.write(f"{r}\t{desc}\n")

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(triples, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

In [ ]:
train_df.to_csv("data/Irish-Places/train.tsv",sep="\t", index=False, header=False)
test_df.to_csv("data/Irish-Places/test.tsv",sep="\t", index=False, header=False)
valid_df.to_csv("data/Irish-Places/dev.tsv",sep="\t", index=False, header=False)

In [ ]:
!python run_bert_triple_classifier.py --task_name kg --do_train  --do_eval --do_predict --data_dir ./data/Irish-Places --bert_model bert-base-uncased --max_seq_length 20 --train_batch_size 32 \
--learning_rate 5e-5 --num_train_epochs 3.0 --output_dir ./output_Irish-Places-data/  --gradient_accumulation_steps 1 --eval_batch_size 512

09/17/2025 10:24:55 - INFO - __main__ -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
09/17/2025 10:24:56 - WARNING - pytorch_pretrained_bert.tokenization -   The pre-trained model you are loading is an uncased model but you have set `do_lower_case` to False. We are setting `do_lower_case=True` for you but you may want to check this behavior.
09/17/2025 10:24:56 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp04bx3m1y
100% 231508/231508 [00:00<00:00, 3088067.17B/s]
09/17/2025 10:24:57 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp04bx3m1y to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
09/17/2025 10:24:57 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_

In [ ]:
df = pd.read_csv("data/Irish_populated_places.csv")

In [ ]:
df.head(1)

,place,placeName,abstract,country,subject,broaderSubject,placeType,broaderPlaceType,population
0,"http://dbpedia.org/resource/Holywell,_Swords","Holywell, Swords",Holywell (Irish: Tobar Naofa) is a neighbourho...,http://dbpedia.org/resource/Republic_of_Ireland,http://dbpedia.org/resource/Category:Neighbour...,"http://dbpedia.org/resource/Category:Swords,_D...","http://dbpedia.org/resource/Swords,_Dublin",http://dbpedia.org/ontology/Place,2479


In [ ]:
df['subject'] = df['subject'].str.replace(
    'http://dbpedia.org/resource/Category:', '', regex=False
).map(lambda x: x.replace('_',' '))

In [ ]:
df['broaderSubject'] = df['broaderSubject'].fillna('').str.replace(
    'http://dbpedia.org/resource/Category:', '', regex=False
).str.replace('_', ' ')

In [ ]:
df['country'] = df['country'].fillna('').str.replace(
    'http://dbpedia.org/resource/', '', regex=False
).str.replace('_', ' ')

In [ ]:
df['placeType'] = df['placeType'].fillna('').str.replace(
    'http://dbpedia.org/resource/', '', regex=False
).str.replace('_', ' ')

In [ ]:
df['broaderPlaceType'] = df['broaderPlaceType'].fillna('').str.replace(
    'http://dbpedia.org/ontology/', '', regex=False
)

In [ ]:
agg_df = df.groupby("place", as_index=False).agg({
    "placeName": "first",               # assume same
    "abstract": "first",                # assume same
    "country": "first",                 # assume same
    "subject": lambda x: "; ".join(set(x.dropna().astype(str))),
    "broaderSubject": lambda x: "; ".join(set(x.dropna().astype(str))),
    "placeType": lambda x: "; ".join(set(x.dropna().astype(str))),
    "broaderPlaceType": lambda x: "; ".join(set(x.dropna().astype(str))),
    "population": "first"               # assume same
})

In [ ]:
agg_df.head(10)

,place,placeName,abstract,country,subject,broaderSubject,placeType,broaderPlaceType,population
0,http://dbpedia.org/resource/Abbeyleix,Abbeyleix,Abbeyleix (/ˈæbiːˈliːks/; Irish: Mainistir Lao...,Republic of Ireland,Towns and villages in County Laois; Planned co...,Townlands of the Republic of Ireland by county...,,,1770
1,http://dbpedia.org/resource/Achill_Island,Achill Island,"Achill Island (/ˈækəl/; Irish: Acaill, Oileán ...",Republic of Ireland,Gaeltacht places in County Mayo; Achill Island...,Geography of County Mayo; Islands of County Ma...,,,2569
2,http://dbpedia.org/resource/Achillbeg,Achillbeg,Acaill Bheag (Achillbeg) is a small island in ...,Republic of Ireland,Villages in Achill Island; Achill Island; Isla...,Achill Island; Islands of County Mayo; Ghost t...,,,1
3,http://dbpedia.org/resource/Aghamore,Aghamore,"Aghamore (Irish: Achadh Mór, meaning 'the grea...",Republic of Ireland,Towns and villages in County Leitrim,Geography of County Leitrim; Towns and village...,,,620
4,http://dbpedia.org/resource/Aglish,Aglish,"Aglish (Irish: An Eaglais, meaning 'the church...",Republic of Ireland,Towns and villages in County Waterford,Geography of County Waterford; Towns and villa...,,,333
5,http://dbpedia.org/resource/Annacotty,Annacotty,"Annacotty (Irish: Áth an Choite, meaning 'the ...",Republic of Ireland,Limerick (city); Towns and villages in County ...,Towns and villages in the Republic of Ireland ...,,,2930
6,http://dbpedia.org/resource/Annagry,Annagry,Anagaire (anglicised as Annagry) is a village ...,Republic of Ireland,Gaeltacht places in County Donegal; The Rosses...,Wikipedia categories named after districts of ...,,,236
7,http://dbpedia.org/resource/Ardnacrusha,Ardnacrusha,Ardnacrusha (Irish: Ard na Croise) is a villag...,Republic of Ireland,Populated places on the River Shannon; Towns a...,Towns and villages in the Republic of Ireland ...,,,1414
8,http://dbpedia.org/resource/Ardoileán,Ardoileán,"Ardoileán or Ard Oileán, known in English as H...",Republic of Ireland,Archaeological sites in County Galway; Islands...,Islands of Ireland; Uninhabited islands by cou...,,,0
9,http://dbpedia.org/resource/Arranmore,Arranmore,Árainn Mhór (English name: Arranmore) is an is...,Republic of Ireland,Gaeltacht places in County Donegal; The Rosses...,Wikipedia categories named after districts of ...,,,469


In [ ]:
def entity_to_text(row):
    text = f"{row['placeName']}. {row['abstract']}. Country: {row['country']}. "
    text += f"category of the place: {row['subject']}. "
    text += f"parent category: {row['broaderSubject']}. "
    text += f"Place types: {row['placeType']}. "
    text += f"Broader place types: {row['broaderPlaceType']}. "
    text += f"Population: {row['population']}"
    return text

agg_df['entity_text'] = agg_df.apply(entity_to_text, axis=1)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("output_Irish-Places-data")  # replace with KG-BERT checkpoint
model = BertModel.from_pretrained("output_Irish-Places-test")
model.eval()  # evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


entity_repr = {}

for idx, row in tqdm(agg_df.iterrows(), total=len(agg_df), desc="Encoding entities"):
    text = row['entity_text']
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()  # tensor [D]
        entity_repr[row['place']] = cls_embedding.cpu()  # store in dict

Encoding entities: 100%|██████████| 296/296 [04:59<00:00,  1.01s/it]


In [ ]:
def compute_similar_places(entity_repr, top_k=10):
    entities = list(entity_repr.keys())
    all_embeddings = torch.stack([entity_repr[e] for e in entities])  # [N, D]
    sims = F.cosine_similarity(all_embeddings.unsqueeze(1), all_embeddings.unsqueeze(0), dim=-1)

    similar_places_dict = {}
    for i, e in enumerate(entities):
        sim_scores = sims[i]
        topk = torch.topk(sim_scores, k=top_k+1)  # +1 to skip itself
        indices = topk.indices.tolist()
        similar_entities = [entities[j] for j in indices if j != i][:top_k]
        similar_places_dict[e] = similar_entities
    return similar_places_dict

In [ ]:
similar_places = compute_similar_places(entity_repr, top_k=10)

In [ ]:
ground_truth = pd.read_csv("./data/ground_truth.csv")
y_true_dict = ground_truth.groupby('geographical_entity_1')['geographical_entity_2'].apply(list).to_dict()

In [ ]:
y_true_list = [
    (e1, e2, score)
    for e1, e2, score in ground_truth[['geographical_entity_1', 'geographical_entity_2', 'F-score']].values
]

In [ ]:
def evluation(k, y_true_dict, similar_places_dict):
  invalid_places = []
  # Compute metrics
  precisions, recalls, ndcgs, hits, map_scores,mrrs = [], [], [], [], [], []
  place_idxs = list(y_true_dict.keys())

  for pid in place_idxs:
      if pid not in similar_places_dict or len(similar_places_dict[pid]) < k:
          invalid_places.append(pid)
          continue
      pred_list, rel_set = similar_places_dict[pid][:k], y_true_dict[pid]
      pred_real = "pid:"+str(pid)+' '+"pred_list:"+str(pred_list)+' '+"rel_set:"+str(rel_set)

      if len(pred_list) == 0:
          continue


      dcg = 0.0
      hit_num = 0.0
      for i in range(len(pred_list)):
          if pred_list[i] in rel_set:
              dcg += 1. / (log(i + 2) / log(2))
              hit_num += 1
      # idcg
      idcg = 0.0
      for i in range(min(len(rel_set), len(pred_list))):
          idcg += 1. / (log(i + 2) / log(2))
      ndcg = dcg / idcg
      recall = hit_num / len(rel_set)
      precision = hit_num / len(pred_list)
      hit = 1.0 if hit_num > 0.0 else 0.0

      #map
      map_score = 0.0
      num_hits = 0.0
      score = 0.0
      for i,p in enumerate(pred_list):
          if p in rel_set and p not in pred_list[:i]:
              num_hits+=1.0
              score+=num_hits/(i+1.0)
      map_score = score/min(len(rel_set),k)
      #map_score = score / min(len(rel_set), len(pred_list)) if len(rel_set) > 0 else 0.0

      #MRR
      rr = 0.0
      for i, p in enumerate(pred_list):
        if p in rel_set:
          rr = 1.0 / (i + 1.0)
          break

      ndcgs.append(ndcg)
      recalls.append(recall)
      precisions.append(precision)
      hits.append(hit)
      map_scores.append(map_score)
      mrrs.append(rr)

  avg_precision = np.mean(precisions) * 100
  avg_recall = np.mean(recalls) * 100
  avg_ndcg = np.mean(ndcgs) * 100
  avg_hit = np.mean(hits) * 100
  avg_map = np.mean(map_scores) * 100
  avg_mrr = np.mean(mrrs) * 100

  print("invalid places:", str(len(invalid_places)))
  print('MAP={:.3f} | NDCG={:.3f} |  Recall={:.3f} | Precision={:.3f} | Hits={:.3f} | MRR={:.3f}'.format(
          avg_map, avg_ndcg, avg_recall, avg_precision, avg_hit, avg_mrr))

In [ ]:
evluation(k=10, y_true_dict=y_true_dict, similar_places_dict=similar_places)

invalid places: 0
MAP=9.818 | NDCG=19.890 |  Recall=8.837 | Precision=17.270 | Hits=78.840 | MRR=43.614
